In [23]:
import numpy as np
import scipy
from scipy import sparse
from scipy.sparse import linalg
import functions as f
from importlib import reload
reload(f)

<module 'functions' from '/Users/keisukemurota/Documents/todo/worms/python/functions.py'>

### ladder model without unitary transformation

In [69]:
I = np.identity(2)
Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2


Sz = sparse.csr_matrix(Sz)
Sx = sparse.csr_matrix(Sx)
Sy = sparse.csr_matrix(Sy)
I = sparse.csr_matrix(I)

Sz1 = sparse.kron(I,Sz,format='csr')
Sz2 = sparse.kron(Sz,I,format='csr')
SzSz = sparse.kron(Sz,Sz,format='csr')

Sx1 = sparse.kron(I,Sx,format='csr')
Sx2 = sparse.kron(Sx,I,format='csr')
SxSx = sparse.kron(Sx,Sx,format='csr')

Sy1 = sparse.kron(I,Sy,format='csr')
Sy2 = sparse.kron(Sy,I,format='csr')
SySy = sparse.kron(Sy,Sy,format='csr')


h = SzSz + SxSx + SySy

tmp = [[0, 2, 0], [1, 3, 0], [0, 1, 1], [0, 3, 2], [1, 2, 2], [2, 4, 0], [3, 5, 0], [2, 3, 1], [2, 5, 2], [3, 4, 2], [4, 6, 0], [5, 7, 0], [4, 5, 1], [4, 7, 2], [5, 6, 2], [6, 0, 0], [7, 1, 0], [6, 7, 1], [6, 1, 2], [7, 0, 2]]
J = [1,2,1]
bonds = np.array(tmp)[:,:2]
b_type = np.array(tmp)[:,2]
L = np.max(bonds)+1

In [70]:
H = sparse.csr_matrix((1<<L,1<<L),dtype=np.float64)
for i in range(len(bonds)):
    H += f.l2nl(h, L, bond = bonds[i], nls=1) * J[b_type[i]]
E_, V_ = scipy.linalg.eigh(H.toarray())
print(E_)

[-6.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
 -4.00000000e+00 -4.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000

### with local unitary transformation

$$
\mathbf{T}_i = \mathbf{S}_i^1 + \mathbf{S}_i^2 \\
\mathbf{D}_i = \mathbf{S}_i^1 - \mathbf{S}_i^2
$$

$$
H = \sum_i J_\perp (\frac{\mathbf{T}_i^2}{2} - 3/4) + \frac{J_\parallel + J_\times}{2} \mathbf{T}_i \cdot \mathbf{T}_{i+1} + \frac{J_\parallel - J_\times}{2} \mathbf{D}_i \cdot \mathbf{D}_{i+1}
$$

In [104]:
Tz = Sz1+Sz2
Tx = Sx1+Sx2
Ty = Sy1+Sy2

Dz = Sz1-Sz2
Dx = Sx1-Sx2
Dy = Sy1-Sy2

TzTz = sparse.kron(Tz,Tz,format='csr')
TxTx = sparse.kron(Tx,Tx,format='csr')
TyTy = sparse.kron(Ty,Ty,format='csr')

DzDz = sparse.kron(Dz,Dz,format='csr')
DxDx = sparse.kron(Dx,Dx,format='csr')
DyDy = sparse.kron(Dy,Dy,format='csr')

Tz2 = Tz@Tz + Tx@Tx + Ty@Ty
h1 = TzTz + TxTx + TyTy
h2 = DzDz + DxDx + DyDy
h0 = Tz2/2 - 3/4*sparse.identity(4)

In [106]:
L = 4
H = sparse.csr_matrix((1<<(2*L),1<<(2*L)),dtype=np.float64)
for i in range(L):
    H += f.l2nl(h1, L, bond = [i,(i+1)%L], nls=2) * (J[0]+J[2])/2.0
    H += f.l2nl(h2, L, bond = [i,(i+1)%L], nls=2) * (J[0]-J[2])/2.0
    H += f.l2nl(h0, L, bond = [i], nls=2) * J[1]
E_, V_ = scipy.linalg.eigh(H.toarray())
print(E_)

/Users/keisukemurota/miniconda3/envs/research/lib/python3.7/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


[-6.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00 -4.00000000e+00
 -4.00000000e+00 -4.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00 -3.00000000e+00
 -3.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000

In [109]:
print(h2)

  (1, 4)	(0.5+0j)
  (1, 8)	(-0.5+0j)
  (2, 4)	(-0.5+0j)
  (2, 8)	(0.5+0j)
  (3, 5)	(-0.5+0j)
  (3, 6)	(0.5+0j)
  (3, 9)	(0.5+0j)
  (3, 10)	(-0.5+0j)
  (4, 1)	(0.5+0j)
  (4, 2)	(-0.5+0j)
  (5, 3)	(-0.5+0j)
  (5, 5)	(1+0j)
  (5, 12)	(-0.5+0j)
  (6, 3)	(0.5+0j)
  (6, 6)	(-1+0j)
  (6, 12)	(0.5+0j)
  (7, 13)	(0.5+0j)
  (7, 14)	(-0.5+0j)
  (8, 1)	(-0.5+0j)
  (8, 2)	(0.5+0j)
  (9, 3)	(0.5+0j)
  (9, 9)	(-1+0j)
  (9, 12)	(0.5+0j)
  (10, 3)	(-0.5+0j)
  (10, 10)	(1+0j)
  (10, 12)	(-0.5+0j)
  (11, 13)	(-0.5+0j)
  (11, 14)	(0.5+0j)
  (12, 5)	(-0.5+0j)
  (12, 6)	(0.5+0j)
  (12, 9)	(0.5+0j)
  (12, 10)	(-0.5+0j)
  (13, 7)	(0.5+0j)
  (13, 11)	(-0.5+0j)
  (14, 7)	(-0.5+0j)
  (14, 11)	(0.5+0j)
